# AH Premium Oscilating Strategy
### Setting up

In [29]:
import backtrader as bt
import pandas as pd
import datetime
import backtrader.feeds as btfeeds
from IPython.core.interactiveshell import InteractiveShell 
import pyecharts.options as opts
from pyecharts.charts import Grid,Line,Page,Scatter,Page,Bar
from pyecharts.faker import Faker
InteractiveShell.ast_node_interactivity = "all"
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
import numpy as np

if_codes = ['IF2110','IF2111','IF2112','IF2201','IF2202','IF2203','IF2204']
ic_codes = ['IC2110','IC2111','IC2112','IC2201','IC2202','IC2203','IC2204']
cn_codes = ['CN2111','CN2112','CN2201','CN2202','CN2203','CN2204']
if_data = pd.read_excel('data\IF.xlsx', sheet_name = if_codes, index_col = 0,skiprows = [0,1],parse_dates=True)
ic_data = pd.read_excel('data\IC.xlsx', sheet_name = ic_codes, index_col = 0,skiprows = [0,1],parse_dates=True)
cn_data = pd.read_excel('data\CN.xlsx', sheet_name = cn_codes, index_col = 0,skiprows = [0,1],parse_dates=True)
csi300 = pd.read_excel('data\CSI300.xlsx', index_col = 0,skiprows = [0,1],parse_dates=True)
csi500 = pd.read_excel('data\CSI500.xlsx', index_col = 0,skiprows = [0,1],parse_dates=True)
a50 = pd.read_excel('data\A50.xlsx', index_col = 0,skiprows = [0,1],parse_dates=True)


In [35]:
for k in if_data.keys():
    if_data[k] = if_data[k].between_time('13:00','11:30').between_time('9:30','14:57')
    if_data[k]['volume'] = 1000000
    if_data[k]['oi'] = 1000000
for k in ic_data.keys():
    ic_data[k] = ic_data[k].between_time('13:00','11:30').between_time('9:30','14:57')
    ic_data[k]['volume'] = 1000000
    ic_data[k]['oi'] = 1000000
for k in cn_data.keys():
    cn_data[k] = cn_data[k].between_time('13:00','11:30').between_time('9:30','14:57')
    cn_data[k]['volume'] = 1000000
    cn_data[k]['oi'] = 1000000
csi300 = csi300.between_time('13:00','11:30').between_time('9:30','14:57')
csi500 = csi500.between_time('13:00','11:30').between_time('9:30','14:57')
a50 = a50.between_time('13:00','11:30').between_time('9:30','14:57')


In [36]:
a50

,Open,Close,High,Low,Volume
Dates,,,,,
2021-10-11 09:30:00,15867.51,15947.78,15955.22,15866.61,0
2021-10-11 09:31:00,15948.22,15977.99,15982.98,15944.15,0
2021-10-11 09:32:00,15978.24,16016.29,16023.56,15973.91,0
2021-10-11 09:33:00,16016.68,16007.15,16022.85,15992.37,0
2021-10-11 09:34:00,16007.02,16038.06,16038.06,16007.02,0
...,...,...,...,...,...
2022-04-22 14:53:00,13512.01,13509.99,13516.66,13507.98,0
2022-04-22 14:54:00,13509.93,13507.27,13511.07,13506.50,0
2022-04-22 14:55:00,13507.13,13504.71,13509.49,13502.67,0


In [14]:
ic_data['IC2111'].resample('1min').last().dropna()

,open,high,low,close
time,,,,
2021-10-11 09:30:00,7035.4,7049.0,7031.2,7036.4
2021-10-11 09:31:00,7036.4,7038.6,7024.0,7024.0
2021-10-11 09:32:00,7023.6,7023.6,7013.0,7017.2
2021-10-11 09:33:00,7018.2,7018.2,6983.4,6996.2
2021-10-11 09:34:00,6996.2,7011.8,6993.0,7001.4
...,...,...,...,...
2021-11-19 14:56:00,7149.6,7149.8,7149.6,7149.8
2021-11-19 14:57:00,7149.6,7149.8,7149.6,7149.8
2021-11-19 14:58:00,7149.8,7149.8,7149.6,7149.6


### Selecting Data

In [ ]:
def relData(data,name):
    n_period = 250
    a = data[['HSAHP']].diff(5)
    b = data[[name]].diff(5)
    chg = a.merge(b,on = 'Dates')
    chg_plot_max = chg.mean() + chg.std()*5
    chg_plot_min = chg.mean() - chg.std()*5
    chg_plot = chg.loc[ (chg[name] > chg_plot_min[name]) & (chg[name] < chg_plot_max[name])]
    d = chg[name].rolling(n_period).cov(chg['HSAHP'])/chg['HSAHP'].rolling(n_period).var()
    d= d.dropna()
    return chg_plot

In [ ]:
stock_data['申万宏源']

### Calculating ADF p-Value

In [ ]:
def CalcAdf(data,name): # return ADF p-value dataframe


    tempData = data[[name,'HSAHP']]
    tempData.loc[:,'ma'] = tempData[name].rolling(ma_period).mean()
    tempData.loc[:,'vol'] = tempData[name].rolling(ma_period).std()
    tempData = tempData.dropna()
    # tempData.loc[:,'adf15'] = tempData[name].rolling(15).apply(lambda x: round(adfuller(x)[1],4))
    tempData.loc[:,'adf30'] = tempData[name].rolling(30).apply(lambda x: round(adfuller(x)[1],4))
    # tempData.loc[:,'adf45'] = tempData[name].rolling(45).apply(lambda x: round(adfuller(x)[1],4))
    tempData.loc[:,'adf60'] = tempData[name].rolling(60).apply(lambda x: round(adfuller(x)[1],4))
    tempData.loc[:,'adf90'] = tempData[name].rolling(90).apply(lambda x: round(adfuller(x)[1],4))
    # tempData.loc[:,'adf15_ma{}'.format(adf_ma_period)] = round(tempData['adf15'].rolling(adf_ma_period).mean(),4)
    tempData.loc[:,'adf30_ma{}'.format(adf_ma_period)] = round(tempData['adf30'].rolling(adf_ma_period).mean(),4)
    # tempData.loc[:,'adf45_ma{}'.format(adf_ma_period)] = round(tempData['adf45'].rolling(adf_ma_period).mean(),4)
    tempData.loc[:,'adf60_ma{}'.format(adf_ma_period)] = round(tempData['adf60'].rolling(adf_ma_period).mean(),4)
    tempData.loc[:,'adf90_ma{}'.format(adf_ma_period)] = round(tempData['adf90'].rolling(adf_ma_period).mean(),4)
    tempData=tempData.dropna()
    return tempData

### Marking Stationery Period

In [ ]:
def markOsciDays(tempData):
    critVal = 0.35
    # initializing N
    minNum = 10
    maxFailRate = 0.1
    fitDays = []
    start, end = 0, 0
    prev = 1
    count = 0
    downcount = 0
    adf_ma_period = 10
    for idx, row in tempData.iterrows():
        # current value fit condition
        if row['adf60_ma{}'.format(adf_ma_period)] < critVal :
            end = idx
    
            if prev > critVal and count == 0:
                start = idx
            count = count + 1

        else: # current value failed condition
            # if in stationery trend
            if count > 0:
                count = count + 1
                downcount = downcount + 1 
            # check if downcount reaches condition to fail it ==> decide whether to record it
            if downcount >= min(count*maxFailRate,minNum):
                if  count >= minNum:
                    fitDays.append((str(start.date()), str(end.date())))
                count = 0
                downcount = 0
        prev = row['adf60_ma{}'.format(adf_ma_period)]
    return fitDays
    btDays = [pd.to_datetime(dp) for dp in fitDays]
###
#fit_days = {}


# Backtesting
### Indicator Definition

In [ ]:
class AdfIndicator(bt.Indicator):
    lines = ('adf1','adf1ma')
    plotinfo = dict(plot=True,
                subplot=True,
           )
    adfdata = None
    columns = None
    def __init__(self,d):
        self.adfdata = d
        self.columns = d.columns
    def next(self):
        dt = self.datas[0].datetime.date(0)        
        if self.adfdata is not None:
            self.lines.adf1[0] = self.adfdata.loc[dt,self.columns[4]]
            self.lines.adf1ma[0] = self.adfdata.loc[dt,self.columns[7]]
            
  
class TrixCritVal(bt.Indicator):
    lines = ('TRIX','TRIX_MA','TRIX_TOP','TRIX_BOT')
    params = (('tperiod', 10),('maperiod',10),('devfactor', 3.0),('movav', bt.indicators.MovAv.Simple),)
    plotinfo = dict(plot=True,
                subplot=True,
           )
    def __init__(self):
        self.l.TRIX = bt.indicators.Trix(period = self.p.tperiod)
        self.l.TRIX_MA =  bt.indicators.MovAv.Simple(self.TRIX, period=self.p.maperiod)
        self.l.TRIX_TOP =  self.p.devfactor*bt.indicators.MaxN(
                                bt.indicators.StandardDeviation(self.l.TRIX ,period=self.p.maperiod),
                                period = 3*self.p.maperiod
                            )
                        

        self.l.TRIX_BOT =  -self.p.devfactor*bt.indicators.MaxN(
                                bt.indicators.StandardDeviation(self.l.TRIX ,period=self.p.maperiod),
                                period = 3*self.p.maperiod
                            )
    def netx(self):
        self.log(self.l.TRIX[0])               
class TrixMomentumCrit(bt.Indicator):
    lines = ('TRIX','TRIX_MMT','MMT_TOP','MMT_BOT')
    plotlines = dict(TRIX=dict(_plotskip=True))
    params = (('tperiod',30),('mmtperiod', 3),('stdperiod',10))
    plotinfo = dict(plot=True,
                subplot=True,
           )
    def __init__(self):
        self.l.TRIX = bt.indicators.Trix(period = self.p.tperiod)
        self.l.TRIX_MMT = bt.indicators.Momentum(self.l.TRIX,period = self.p.mmtperiod)
        self.l.MMT_TOP = bt.indicators.StandardDeviation(self.l.TRIX_MMT ,period=self.p.stdperiod)
        self.l.MMT_BOT = -bt.indicators.StandardDeviation(self.l.TRIX_MMT ,period=self.p.stdperiod)
    def next(self):
        # print(self.l.TRIX[0])
        pass

# class portValue(bt.Indicator):
#     lines = ('value',)
#     this = None
#     prev = None
#     def __init__(self):
#         this =
#     def next(self):
#         self.l.value[0] = self.broker.get_value()


### Strategy Definition

In [ ]:
class BollRev(bt.Strategy):

    params = (
        ('period',20),
        ('numDevs',1.5),
        ('TrixObsPeriod',10),
        ('TrixDevFactor',1),
        ('logname','log.csv'),
    )
    tradeLog = None
    osciDays = None
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            ops = None
            if order.isbuy():
                #空头平仓
                if self.position.size == 0:
                    ops = 'BuyToCover'
                else:
                    ops = 'LongOpen'
            elif order.issell():
                # 多头平仓
                if self.position.size == 0:
                    ops = 'SellClose'
                else:
                    ops = 'ShortSell'
            self.tradeLog.write('%s,%s,%.2f,%s,%.2f\n' % (   self.data.num2date(order.executed.dt).date(),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl)
                                                    )
            self.bar_executed = len(self)

        # elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            # self.log('  **********  Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    # def start(self):
    #     self.tradeLog = open('res/'+self.p.logFile, 'w+')
    #     self.mystats.write('Dates,Trade,Price,Size\n')

    def __init__(self,adfdata,osciD):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Dates,Trade,Price,Size,PnL\n')
        self.dataopen = self.datas[0].open
        self.boll = bt.indicators.BollingerBands(period = self.p.period,
                                                   devfactor = self.p.numDevs)
        self.TRIMA = bt.indicators.TripleExponentialMovingAverage(period=self.p.period)
        # self.ADF = AdfIndicator(d = adfdata)
        self.TRIX = TrixCritVal(tperiod = 30,maperiod =15,devfactor = 1.5)
        self.TRIX.csv=True
        self.TRIX_MMT = TrixMomentumCrit(tperiod=30,mmtperiod = 3,stdperiod = 20)
        self.osciDays = osciD
        self.LongOnly = (self.TRIX.TRIX >= self.TRIX.TRIX_TOP) * (self.TRIX_MMT.TRIX_MMT >= self.TRIX_MMT.MMT_TOP)
        self.LongOnly.csv = True
        self.ShortOnly = (self.TRIX.TRIX <= self.TRIX.TRIX_BOT) * (self.TRIX_MMT.TRIX_MMT <= self.TRIX_MMT.MMT_BOT)
        self.ShortOnly.csv = True
        self.CrossTop = 0 # 1：向上突破， -1：向下突破， 0：不相交
        self.CrossBot = 0
        self.order = None
    def next_open(self):

        # Marking Crossing Over
        if self.dataopen[-1]<self.boll.top[-1] and self.dataopen[0]> self.boll.top[0]:
            self.CrossTop =  1 
        elif self.dataopen[-1]>self.boll.top[-1] and self.dataopen[0]< self.boll.top[0]:
            self.CrossTop =  -1
        else:
            self.CrossTop = 0

        if self.dataopen[-1]<self.boll.bot[-1] and self.dataopen[0]> self.boll.bot[0]:
            self.CrossBot =  1
        elif self.dataopen[-1]>self.boll.bot[-1] and self.dataopen[0]< self.boll.bot[0]:
            self.CrossBot = -1
        else:
            self.CrossBot = 0   

        
        # Holding Long position
        if self.position.size > 0:
            # ----- 多头平仓:   1. 突破上界   2.逆势砍仓
            if self.CrossTop > 0:   # 向上突破
                self.order = self.sell(exectype=bt.Order.Limit,price=self.dataopen[0])
            else: # 未突破上界，检查是否逆趋势
                if self.ShortOnly[0] and not self.tradeOsci(): # 如果处于下跌趋势 & 不在震荡日期，多单平仓
                    self.order = self.sell(exectype=bt.Order.Limit,price=self.dataopen[0])
        
        if self.position.size < 0:
            # ----- 空头平仓:   1. 突破下界   2.逆势砍仓
            if self.CrossBot <  0:   # 突破下界
                self.order = self.buy(exectype=bt.Order.Limit,price=self.dataopen[0])
            else: # 未突破上界，检查是否逆趋势
                if self.LongOnly[0] and not self.tradeOsci(): # 如果处于上升趋势 & 不在震荡日期，空单平仓
                    self.order = self.buy(exectype=bt.Order.Limit,price=self.dataopen[0])


        # No position
        if self.position.size == 0:
            # Oscilating Market ==> can do both Long/Short
            if self.tradeOsci():
                if self.CrossTop > 0:
                    self.order = self.sell(exectype=bt.Order.Limit,price=self.dataopen[0])
                if self.CrossBot < 0:
                    self.order = self.buy(exectype=bt.Order.Limit,price=self.dataopen[0])
            else: # Chasing Trend ==> only follow the trend
                if self.ShortOnly[0] and self.CrossTop > 0:   # Down trend & crossed Upper Boll Band
                    self.order = self.sell(exectype=bt.Order.Limit,price=self.dataopen[0])
                if self.LongOnly[0] and self.CrossBot < 0:    # Up trend & crossed lower Boll Band
                    self.order = self.buy(exectype=bt.Order.Limit,price=self.dataopen[0])

    def stop(self):
        self.tradeLog.close()
        
    # True if Oscilating, can do both side
    def tradeOsci(self):
        for interval in self.osciDays:
            if interval[0]<=pd.to_datetime(str(self.datas[0].datetime.date(0)))<=interval[1]:
                return True
        return False



In [ ]:
temp

In [ ]:
class BollRevTrend(bt.Strategy):

    params = (
        ('period',20),
        ('numDevs',1.5),
        ('TrixObsPeriod',10),
        ('TrixDevFactor',1),
        ('logname','log.csv'),
    )
    tradeLog = None
    osciDays = None
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            ops = None
            if order.isbuy():
                #空头平仓
                if self.position.size == 0:
                    ops = 'BuyToCover'
                else:
                    ops = 'LongOpen'
            elif order.issell():
                # 多头平仓
                if self.position.size == 0:
                    ops = 'SellClose'
                else:
                    ops = 'ShortSell'
            self.tradeLog.write('%s,%s,%.2f,%s,%.2f\n' % (   self.data.num2date(order.executed.dt).date(),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl)
                                                    )
            self.bar_executed = len(self)

        # Write down: no pending order
        self.order = None


    def __init__(self,adfdata,osciD):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Dates,Trade,Price,Size,PnL\n')
        self.dataopen = self.datas[0].open
        self.boll = bt.indicators.BollingerBands(period = self.p.period,
                                                   devfactor = self.p.numDevs)
        self.TRIMA = bt.indicators.TripleExponentialMovingAverage(period=self.p.period)
        # self.ADF = AdfIndicator(d = adfdata)
        self.TRIX = TrixCritVal(tperiod = 30,maperiod =15,devfactor = 1.5)
        self.TRIX.csv=True
        self.TRIX_MMT = TrixMomentumCrit(tperiod=30,mmtperiod = 3,stdperiod = 20)
        self.osciDays = osciD
        self.LongOnly = (self.TRIX.TRIX >= self.TRIX.TRIX_TOP) * (self.TRIX_MMT.TRIX_MMT >= self.TRIX_MMT.MMT_TOP)
        self.LongOnly.csv = True
        self.ShortOnly = (self.TRIX.TRIX <= self.TRIX.TRIX_BOT) * (self.TRIX_MMT.TRIX_MMT <= self.TRIX_MMT.MMT_BOT)
        self.ShortOnly.csv = True
        self.CrossTop = 0 # 1：向上突破， -1：向下突破， 0：不相交
        self.CrossBot = 0
        self.CrossMid = 0
        self.order = None
    def next_open(self):

        # Marking Crossing Over
        if self.dataopen[-1]<self.boll.top[-1] and self.dataopen[0]> self.boll.top[0]:
            self.CrossTop =  1 
        elif self.dataopen[-1]>self.boll.top[-1] and self.dataopen[0]< self.boll.top[0]:
            self.CrossTop =  -1
        else:
            self.CrossTop = 0

        if self.dataopen[-1]<self.boll.bot[-1] and self.dataopen[0]> self.boll.bot[0]:
            self.CrossBot =  1
        elif self.dataopen[-1]>self.boll.bot[-1] and self.dataopen[0]< self.boll.bot[0]:
            self.CrossBot = -1
        else:
            self.CrossBot = 0   
        
        if self.dataopen[-1]<self.boll.mid[-1] and self.dataopen[0]> self.boll.mid[0]:
            self.CrossMid =  1
        elif self.dataopen[-1]>self.boll.mid[-1] and self.dataopen[0]< self.boll.mid[0]:
            self.CrossMid = -1
        else:
            self.CrossMid = 0   

        
        # Holding Long position
        if self.position.size > 0:
            # ----- 多头平仓:   1. 突破上界   2.逆势砍仓
            if self.CrossTop > 0:   # 向上突破
                self.order = self.sell(exectype=bt.Order.Limit,price=self.dataopen[0])
            else: # 未突破上界，检查是否逆趋势
                if self.ShortOnly[0] and not self.tradeOsci(): # 如果处于下跌趋势 & 不在震荡日期，多单平仓
                    self.order = self.sell(exectype=bt.Order.Limit,price=self.dataopen[0])
        
        if self.position.size < 0:
            # ----- 空头平仓:   1. 突破下界   2.逆势砍仓
            if self.CrossBot <  0:   # 突破下界
                self.order = self.buy(exectype=bt.Order.Limit,price=self.dataopen[0])
            else: # 未突破上界，检查是否逆趋势
                if self.LongOnly[0] and not self.tradeOsci(): # 如果处于上升趋势 & 不在震荡日期，空单平仓
                    self.order = self.buy(exectype=bt.Order.Limit,price=self.dataopen[0])


        # No position
        if self.position.size == 0:
            # Oscilating Market ==> can do both Long/Short
            if self.tradeOsci():
                if self.CrossTop > 0:
                    self.order = self.sell(exectype=bt.Order.Limit,price=self.dataopen[0])
                if self.CrossBot < 0:
                    self.order = self.buy(exectype=bt.Order.Limit,price=self.dataopen[0])
            else: # Chasing Trend ==> only follow the trend
                
                if self.ShortOnly[0] and (self.CrossTop > 0 or self.CrossMid != 0):   # Down trend & crossed Upper Boll Band
                    # self.log('Short, {}, {}'.format(self.CrossTop,self.CrossMid))
                    self.order = self.sell(exectype=bt.Order.Limit,price=self.dataopen[0])
                if self.LongOnly[0] and (self.CrossBot < 0 or self.CrossMid != 0):    # Up trend & crossed lower Boll Band
                    # self.log('Long, {}, {}'.format(self.CrossTop,self.CrossMid))
                    self.order = self.buy(exectype=bt.Order.Limit,price=self.dataopen[0])

    def stop(self):
        self.tradeLog.close()
        
    # True if Oscilating, can do both side
    def tradeOsci(self):
        for interval in self.osciDays:
            if interval[0]<=pd.to_datetime(str(self.datas[0].datetime.date(0)))<=interval[1]:
                return True
        return False



### Do backtest

In [ ]:
import backtrader as bt
import numpy as np
import bokeh
import backtrader.strategies as btstrats
import backtrader.analyzers as btanalyzers

def runBacktest(sdata,name,fitDays):
    btDays = [pd.to_datetime(dp) for dp in fitDays]
    tempData = sdata
    cerebro = bt.Cerebro()
    cerebro.broker = bt.brokers.BackBroker(slip_perc=0.001)  # 0.5%
    cerebro.broker.set_shortcash(False)
    cerebro.broker.setcommission(commission=0)
    cerebro.broker.setcash(100000.0)


    bt_data = tempData[[name]]
    bt_data.index.name = 'Dates'
    bt_data.columns = ['Close']

    bt_data.loc[:,'Open'] = bt_data.loc[:,'Close']
    bt_data.loc[:,'High'] = bt_data.loc[:,'Close']
    bt_data.loc[:,'Low'] = bt_data.loc[:,'Close']
    bt_data.loc[:,'Volume'] = 1000000
    datafeed = bt.feeds.PandasData(dataname = bt_data, timeframe=bt.TimeFrame.Days)
    cerebro.adddata(datafeed, name=name)
    cerebro.addstrategy(BollRev,adfdata = tempData,osciD = btDays,logname = name+'.csv')

    # cerebro.addobserver(portValue)
    cerebro.broker.getvalue()
    cerebro.addsizer(bt.sizers.AllInSizerInt) 
    cerebro.addwriter(bt.WriterFile, out = 'res\{}_res.csv'.format(name),csv=True)
    # cerebro.broker.plotinfo(plot = False)
    cerebro.run(runonce = True,cheat_on_open=True)
    
    return 0
def runTrend(sdata,name,fitDays):
    btDays = [pd.to_datetime(dp) for dp in fitDays]
    tempData = sdata
    cerebro = bt.Cerebro()
    cerebro.broker = bt.brokers.BackBroker(slip_perc=0.001)  # 0.5%
    cerebro.broker.set_shortcash(False)
    cerebro.broker.setcommission(commission=0)
    cerebro.broker.setcash(100000.0)


    bt_data = tempData[[name]]
    bt_data.index.name = 'Dates'
    bt_data.columns = ['Close']

    bt_data.loc[:,'Open'] = bt_data.loc[:,'Close']
    bt_data.loc[:,'High'] = bt_data.loc[:,'Close']
    bt_data.loc[:,'Low'] = bt_data.loc[:,'Close']
    bt_data.loc[:,'Volume'] = 1000000
    datafeed = bt.feeds.PandasData(dataname = bt_data, timeframe=bt.TimeFrame.Days)
    cerebro.adddata(datafeed, name=name)
    cerebro.addstrategy(BollRevTrend,adfdata = tempData,osciD = btDays,logname = name+'_trend.csv')

    # cerebro.addobserver(portValue)
    cerebro.broker.getvalue()
    cerebro.addsizer(bt.sizers.AllInSizerInt) 
    cerebro.addwriter(bt.WriterFile, out = 'res\{}_trend_res.csv'.format(name),csv=True)
    # cerebro.broker.plotinfo(plot = False)
    cerebro.run(runonce = True,cheat_on_open=True)
    return cerebro.plot()
    
def readPnL(sname):
    pl = pd.read_csv('res\{}_res.csv'.format(sname),skiprows=[0],encoding='unicode_escape')
    pl = pl[[   'datetime', 'close', 
                    'TRIX', 'TRIX_MA', 
                    'TRIX_TOP','TRIX_BOT', 'cash', 'value',]]
    pl = pl.dropna().set_index('datetime')
    pl.index = pd.to_datetime(pl.index)
    return pl
def readTrendPnL(sname):
    pl = pd.read_csv('res\{}_trend_res.csv'.format(sname),skiprows=[0],encoding='unicode_escape')
    pl = pl[[   'datetime', 'close', 
                    'TRIX', 'TRIX_MA', 
                    'TRIX_TOP','TRIX_BOT', 'cash', 'value',]]
    pl = pl.dropna().set_index('datetime')
    pl.index = pd.to_datetime(pl.index)
    return pl



In [ ]:
def tradeMark(trade_date,trade_dir,trade_price):
    if trade_dir == 'ShortSell':
        return opts.MarkPointItem(  coord=[trade_date.date(),trade_price],
                                    symbol = 'diamond',
                                    symbol_size = 12,
                                    itemstyle_opts = opts.ItemStyleOpts(color = 'green') )
    if trade_dir == 'SellClose':
        return opts.MarkPointItem(  coord=[trade_date.date(),trade_price],
                                    symbol = 'circle',
                                    symbol_size = 10,
                                    itemstyle_opts = opts.ItemStyleOpts(color = 'green') )
    if trade_dir == 'LongOpen':
        return opts.MarkPointItem(  coord=[trade_date.date(),trade_price],
                                    symbol = 'diamond',
                                    symbol_size = 12,
                                    itemstyle_opts = opts.ItemStyleOpts(color = 'red') )
    if trade_dir == 'BuyToCover':
        return opts.MarkPointItem(  coord=[trade_date.date(),trade_price],
                                    symbol = 'circle',
                                    symbol_size = 10,
                                    itemstyle_opts = opts.ItemStyleOpts(color = 'red') )

### Price Chart

In [ ]:
def plotPrice(tempData,name,fitDays,tlog):
    rel = ((100*(tempData[name]+100)/tempData['HSAHP'])-100)
    tradelist = []
    for idx,row in tlog.iterrows():
        tradelist.append(tradeMark(idx,row.Trade,row.Price))
    price_ah = (
        Line(
            )
        .add_xaxis(tempData.index.date.tolist())
        
        .add_yaxis( '溢价', round(tempData[name],2).values.tolist(),
                    yaxis_index = 0,xaxis_index = 0,is_smooth=False,
                    label_opts =  opts.LabelOpts(is_show = False),
                    is_symbol_show = False,
                    color = 'brown',
                    linestyle_opts = opts.LineStyleOpts(width=1.5,type_ = 'solid'),
                    markpoint_opts=opts.MarkPointOpts(
                        data = tradelist,
                    ),
                    
                )
        
        .add_yaxis( 'ma20', round(tempData.ma,2),
                    yaxis_index = 0,xaxis_index = 0,is_smooth=False,
                    label_opts =  opts.LabelOpts(is_show = False),
                    color = 'black',
                    is_symbol_show = False,
                    linestyle_opts = opts.LineStyleOpts(width=1.5,type_ = 'dashed', opacity=0.69),
                    tooltip_opts=opts.TooltipOpts(is_show = False),
                )
        .add_yaxis( 'BollU', round(tempData.ma + tempData.vol*1.5,2),
                    yaxis_index = 0,xaxis_index = 0,is_smooth=False,
                    label_opts =  opts.LabelOpts(is_show = False),
                    color = 'purple',
                    is_symbol_show = False,
                    linestyle_opts = opts.LineStyleOpts(width=1.5,type_ = 'solid',opacity=0.7),
                    tooltip_opts=opts.TooltipOpts(is_show = False),
                )
        .add_yaxis( 'BollL', round(tempData.ma - tempData.vol*1.5,2),
                    yaxis_index = 0,xaxis_index = 0,is_smooth=False,
                    label_opts =  opts.LabelOpts(is_show = False),
                    color = 'green',
                    is_symbol_show = False,
                    linestyle_opts = opts.LineStyleOpts(width=1.5,type_ = 'solid',opacity=0.7),
                    tooltip_opts=opts.TooltipOpts(is_show = False),
                )
        
        .extend_axis(yaxis=opts.AxisOpts(name = 'HSAHP',
                                        type_="value", 
                                        position="right",
                                        name_rotate = 45,
                                        min_interval = 1,
                                        max_ = 'dataMax',
                                        min_ = 'dataMin',
                                        )
                    )
        .add_yaxis("HSAHP", round(tempData['HSAHP'],1),
                yaxis_index = 1,xaxis_index = 0,
                label_opts =  opts.LabelOpts(is_show = False),
                color = 'red',
                is_symbol_show = False,
                linestyle_opts = opts.LineStyleOpts(width=1, opacity=0.7),)
        .extend_axis(yaxis=opts.AxisOpts(name = '相对溢价',
                                        type_="value", 
                                        position="left",
                                        name_rotate = 45,
                                        offset=42,
                                        min_interval = 1,
                                        max_ = 'dataMax',
                                        min_ = 'dataMin',
                                        )
                    )
        
        .add_yaxis("相对", round(rel,1).values.tolist(),
                yaxis_index = 2,xaxis_index = 0, is_smooth=True,
                color = 'blue',
                is_symbol_show = False,
                label_opts = opts.LabelOpts( is_show = False,position = 'left'),
                linestyle_opts = opts.LineStyleOpts(width=1,opacity=0.3),
                tooltip_opts=opts.TooltipOpts(is_show = False),
                )
        .set_global_opts(
                        title_opts=opts.TitleOpts( title=name+"溢价",
                                                pos_left = 'center'),
                        legend_opts = opts.LegendOpts(
                            pos_top = '7%',pos_left = '20%',
                            ),

                        datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=False, type_="inside", xaxis_index=[0, 0,0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 1,5], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=False, type_="inside",xaxis_index=[0,1,5],#filter_mode = 'weakFilter',
                                ),
                            ],
                        xaxis_opts=opts.AxisOpts(
                            grid_index=0,
    #                         type_ = 'time',
    #                         name="Dates",
                            axislabel_opts=opts.LabelOpts(is_show = True),
                        ),
                        yaxis_opts=opts.AxisOpts(
                            name = '溢价',
                            type_ = 'value',
                            offset=0,
                            name_rotate = 45,
                            min_interval = 5,
                            max_ = 'dataMax',
                            min_ = 'dataMin',
                            splitarea_opts=opts.SplitAreaOpts(
                                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                            )
                        ),
                        tooltip_opts=opts.TooltipOpts(
                            trigger="axis",
                            axis_pointer_type="cross",
                            background_color="rgba(245, 245, 245, 0.8)",
                            border_width=1,
                            border_color="#ccc",
                            textstyle_opts=opts.TextStyleOpts(color="#000"),
                        ),
                        axispointer_opts=opts.AxisPointerOpts(
                            is_show=True, link=[{"xAxisIndex": [0,1,5]},{"yAxisIndex": [0,1,2]},{"yAxisIndex": [5,6]}]
                        ),
                )
        .set_series_opts(
            markarea_opts=opts.MarkAreaOpts(
                
                data=[
                    opts.MarkAreaItem(x=dr,itemstyle_opts=opts.ItemStyleOpts(opacity=0.1)) for dr in fitDays
                ],
            )
        )
        
    )
    return price_ah

In [ ]:
def plotAdf(tempData):
    adf_test = (
                Line()
                .add_xaxis(tempData.index.date.tolist())
                
                .add_yaxis(
                            series_name="adf60",
                            y_axis=tempData['adf60'].values,
                            xaxis_index = 1,
                            yaxis_index = 3,
                            is_symbol_show = False,
                            label_opts=opts.LabelOpts(is_show=False),
                            tooltip_opts=opts.TooltipOpts(is_show = False),
                        )
                .add_yaxis(
                            series_name="adf60_ma{}".format(adf_ma_period),
                            y_axis=tempData["adf60_ma{}".format(adf_ma_period)],
                            xaxis_index = 1,
                            yaxis_index = 3,
                            is_symbol_show = False,
                            label_opts=opts.LabelOpts(is_show=False),
                            tooltip_opts=opts.TooltipOpts(is_show = False),
                        )
                
                .set_global_opts(
                            legend_opts = opts.LegendOpts(is_show=False),
                            xaxis_opts=opts.AxisOpts(
                                grid_index=1,
                                axislabel_opts=opts.LabelOpts(rotate=50,is_show = False),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                type_="value",
                                max_ = 1,
                                min_ = 0,
                                axistick_opts=opts.AxisTickOpts(is_show=False),
                                axislabel_opts=opts.LabelOpts(),
                                splitline_opts=opts.SplitLineOpts(is_show=True),
                            ),
                            tooltip_opts=opts.TooltipOpts(is_show=False),
                        )
                .set_series_opts(
                        label_opts=opts.LabelOpts(is_show=False),
                        markline_opts=opts.MarkLineOpts(
                            data=[
                                opts.MarkLineItem(type_="average", name="平均值"),
                            ]
                        ),
                    )
                )
    return adf_test

### ADF p-value Plot 

### Scatter PLot

In [ ]:
def plotRelChg(chg_plot,name):
    ah_rel_chg = (
        Scatter()
        .add_xaxis(xaxis_data=chg_plot['HSAHP'])
        .add_yaxis(
            series_name="",
            y_axis=chg_plot[name],
            xaxis_index=2,
            yaxis_index=4,
            symbol_size=4,
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            legend_opts=opts.LegendOpts(is_show=False),
            xaxis_opts=opts.AxisOpts(
                type_="value",
                grid_index=2,
                # max_ = 'dataMax',
                # min_ = 'dataMin',
                name='HSAHP变化%',
                axislabel_opts=opts.LabelOpts(rotate=50),
            ),
            yaxis_opts=opts.AxisOpts(
                name=name+'AH溢价变化%',
                type_="value",
                # max_ = 'dataMax',
                # min_ = 'dataMin',
                axistick_opts=opts.AxisTickOpts(is_show=True),
                axislabel_opts=opts.LabelOpts(rotate=50),
                splitline_opts=opts.SplitLineOpts(
                    is_show=True),
            ),
            tooltip_opts=opts.TooltipOpts(is_show=False),
        )
    )
    return ah_rel_chg

### ADF Distribution Plot

In [ ]:
def plotDist(tempData,sname):
    d60 = tempData[['adf60']]
    d60 = d60.groupby(pd.cut(d60["adf60"], np.arange(0, 1.1, 0.1))).count()
    d60.index.name='Range'
    d60ma = tempData[['adf60_ma{}'.format(adf_ma_period)]]
    d60ma = d60ma.groupby(pd.cut(d60ma['adf60_ma{}'.format(adf_ma_period)], np.arange(0, 1.1, 0.1))).count()
    d60ma.index.name='Range'
    ticks = [p/10 for p in range(1, 11)]

    adf_dist = (
                Bar()
                .add_xaxis(ticks)
                .reversal_axis()
                .add_yaxis("adf60", d60.adf60.values.tolist(),
                                xaxis_index = 3,yaxis_index =5,label_opts=opts.LabelOpts(is_show=False)
                                )
                .set_global_opts(
                    legend_opts = opts.LegendOpts(is_show = False),
                    xaxis_opts=opts.AxisOpts(
                                grid_index=3,
                                ),

                )
            )

    adfma_dist = (
                Bar()
                .add_xaxis(ticks)
                .reversal_axis()
                .add_yaxis("adf60_ma{}".format(adf_ma_period), d60ma['adf60_ma{}'.format(adf_ma_period)].values.tolist(),
                                xaxis_index = 4,yaxis_index =6,label_opts=opts.LabelOpts(is_show=False)
                                )
                .set_global_opts(
                    legend_opts = opts.LegendOpts(is_show = False),
                    xaxis_opts=opts.AxisOpts(
                                grid_index=4,
                                ),
                )
            )
    return adf_dist,adfma_dist

### Port Val & Trade History

In [ ]:
def getPortValue(tempData,sname):
    port_value = (
        Line()
        .add_xaxis(tempData.index.date.tolist())
        .add_yaxis(
                    series_name="Osci_Value",
                    y_axis=round((tempData['value']/1000),1).values.tolist(),
                    xaxis_index = 5,
                    yaxis_index = 7,
                    color = 'black',
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    )
        .extend_axis(
                    yaxis=opts.AxisOpts(
                                name = 'PnL',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)

                            ),
                    )
        .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '71%',pos_left = '13%'),
                            xaxis_opts=opts.AxisOpts(
                                grid_index=5,
                                axislabel_opts=opts.LabelOpts(is_show = False),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Value',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=False,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
                            tooltip_opts=opts.TooltipOpts(is_show=True,formatter="{value} K"),
                            visualmap_opts=opts.VisualMapOpts(type_="size", max_=30000, min_=20),
                        )
    )

    pnl_scatter = (
        Scatter()
        .add_xaxis(tempData.index.date.tolist())
        .add_yaxis( "Osci_Win", 0.001*tempData.PnL_Win,
                    xaxis_index = 5,
                    yaxis_index = 8,
                    color = 'rgb(255, 0, 0)',
                    label_opts = opts.LabelOpts( is_show = False),
                    tooltip_opts=opts.TooltipOpts(is_show=False),

                )
        .add_yaxis( "Osci_Loss", 0.001*tempData.PnL_Loss,
                    xaxis_index = 5,
                    yaxis_index = 8,
                    color = 'rgb(64, 191, 64)',
                    label_opts = opts.LabelOpts( is_show = False),
                    tooltip_opts=opts.TooltipOpts(is_show=False),
                )
        .set_global_opts(
                            legend_opts = opts.LegendOpts(is_show = True),
                            xaxis_opts=opts.AxisOpts(
                                grid_index=5, #6
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            # yaxis_opts=opts.AxisOpts(
                            #     name = 'PnL',
                            #     max_ = 'dataMax',
                            #     min_ = 'dataMin',
                            #     axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                            #     splitline_opts=opts.SplitLineOpts(  is_show=True,
                            #                                         linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)

                            # ),
                            
                        )
    )
    # overlap  = port_value.overlap(pnl_scatter)
    return port_value,pnl_scatter


In [ ]:
sname = "东方证券"

rel_chg[sname] = relData(all_data,sname)
stock_data[sname] = CalcAdf(all_data,sname)
fit_days[sname] = markOsciDays(stock_data[sname])
runBacktest(stock_data[sname],sname,fitDays=fit_days[sname])

pnl = readPnL(sname)
# combine stockdata& 
temp = stock_data[sname].loc[pnl.index]
temp = temp.round(4)
temp.index = pd.to_datetime(temp.index)
temp = temp.join(pnl)
# combine stock data and trade history
tradelog = pd.read_csv('tradelog/'+sname+'.csv')
tradelog = tradelog.set_index('Dates')
tradelog.index = pd.to_datetime(tradelog.index)
temp = temp.join(tradelog)
temp = temp.join(tradelog.loc[tradelog.PnL>0],rsuffix= '_Win')
temp = temp.join(tradelog.loc[tradelog.PnL<0],rsuffix = '_Loss')
temp.replace(0, np.nan, inplace=True)
stock_data[sname] = temp
ah_price_plot = plotPrice(stock_data[sname],sname,fit_days[sname],tradelog)
ah_adf_plot = plotAdf(stock_data[sname])
rel_chg_plot = plotRelChg(rel_chg[sname],sname)
adf_dist_plot,adfma_dist_plot = plotDist(stock_data[sname],sname)
portValue = getPortValue(stock_data[sname],sname)
grid = Grid(
            init_opts=opts.InitOpts(
            width="1440px",
            height="600px",
            animation_opts=opts.AnimationOpts(animation=False),
            )
        )
grid.add(ah_price_plot,grid_index = 0,grid_opts = opts.GridOpts(pos_right = '40%',pos_bottom = '50%'),is_control_axis_index = True)
grid.add(ah_adf_plot,grid_index = 1,grid_opts = opts.GridOpts(pos_right = '40%',pos_top = '75%'),is_control_axis_index = True)
grid.add(rel_chg_plot,grid_index = 2,grid_opts = opts.GridOpts(pos_left = '68%',pos_bottom = '35%'),is_control_axis_index = True)
grid.add(adf_dist_plot,grid_index = 3,grid_opts = opts.GridOpts(pos_left = '65%',pos_right='23%',pos_top = '75%'),is_control_axis_index = True)
grid.add(adfma_dist_plot,grid_index = 4,grid_opts = opts.GridOpts(pos_left = '80%',pos_right = '8%',pos_top = '75%'),is_control_axis_index = True)
grid.add(portValue,grid_index = 5,grid_opts = opts.GridOpts(pos_right = '40%',pos_top = '70%',pos_bottom = '46%'),is_control_axis_index = True)
# # grid.add(pnl_scatter,grid_index = 6,grid_opts = opts.GridOpts(pos_left = '65%',pos_top = '69%',pos_bottom = '46%'),is_control_axis_index = True)


grid.render('report/'+sname+'adf.html')

In [ ]:
banklist = all_data.filter(like='银行', axis=1).columns.tolist()
finlist = all_data.filter(like='证券', axis=1).columns.tolist()
finlist.append('国泰君安')
finlist.append('申万宏源')

In [ ]:
grid_tabs = {}
for nm in finlist+banklist:
    sname = nm
    rel_chg[sname] = relData(all_data,sname)
    stock_data[sname] = CalcAdf(all_data,sname)
    fit_days[sname] = markOsciDays(stock_data[sname])
    runBacktest(stock_data[sname],sname,fitDays=fit_days[sname])
    pnl = readPnL(sname)
    # combine stockdata& 
    temp = stock_data[sname].loc[pnl.index]
    temp = temp.round(4)
    temp.index = pd.to_datetime(temp.index)
    temp = temp.join(pnl)
    # combine stock data and trade history
    tradelog = pd.read_csv('tradelog/'+sname+'.csv')
    tradelog = tradelog.set_index('Dates')
    tradelog.index = pd.to_datetime(tradelog.index)
    temp = temp.join(tradelog)
    temp = temp.join(tradelog.loc[tradelog.PnL>0],rsuffix= '_Win')
    temp = temp.join(tradelog.loc[tradelog.PnL<0],rsuffix = '_Loss')
    temp.replace(0, np.nan, inplace=True)
    # stock_data[sname] = temp
    ah_price_plot = plotPrice(temp,sname,fit_days[sname],tradelog)
    ah_adf_plot = plotAdf(temp)
    rel_chg_plot = plotRelChg(rel_chg[sname],sname)
    adf_dist_plot,adfma_dist_plot = plotDist(temp,sname)
    portValue,pnlScatter = getPortValue(temp,sname)
  

    ## test trend

    runTrend(stock_data[sname],sname,fitDays=fit_days[sname])
    cols = [sname, 'HSAHP', 'ma', 'vol', 'adf30', 'adf60', 'adf90', 'adf30_ma10',
       'adf60_ma10', 'adf90_ma10', 'close', 'TRIX', 'TRIX_MA', 'TRIX_TOP',
       'TRIX_BOT', 'cash', 'value',]
    pnl = readTrendPnL(sname)
    # combine stockdata& 
    temp_t = stock_data[sname].loc[pnl.index]
    temp_t = temp_t.round(4)
    temp_t.index = pd.to_datetime(temp_t.index)
    temp_t = temp_t.join(pnl,lsuffix = '_Osci',rsuffix= '_CTA')
    # combine stock data and trade history
    tradelog = pd.read_csv('tradelog/'+sname+'_trend.csv')
    tradelog = tradelog.set_index('Dates')
    tradelog.index = pd.to_datetime(tradelog.index)
    temp_t = temp_t.join(tradelog)
    temp_t = temp_t.join(tradelog.loc[tradelog.PnL>0],rsuffix= '_Win')
    temp_t = temp_t.join(tradelog.loc[tradelog.PnL<0],rsuffix = '_Loss')
    temp_t.replace(0, np.nan, inplace=True)
    tradelist = []
    cols = stock_data[sname].columns.to_list() + ['cash','value']
    stock_data[sname] =  temp[cols].join(temp_t[['cash','value']],lsuffix = '_OSCI',rsuffix = '_CTA')    # stock_data[sname] = temp[cols]

    for idx,row in tradelog.iterrows():
        tradelist.append(tradeMark(idx,row.Trade,row.Price))
    ah_price_plot.add_yaxis( 'CTA', round(temp_t[sname],2).values.tolist(),
                    yaxis_index = 0,xaxis_index = 0,is_smooth=False,
                    label_opts =  opts.LabelOpts(is_show = False),
                    is_symbol_show = False,
                    color = 'brown',
                    linestyle_opts = opts.LineStyleOpts(width=1.5,type_ = 'solid'),
                    markpoint_opts=opts.MarkPointOpts(
                        data = tradelist,
                    ),
                )
    ah_adf_plot = plotAdf(temp_t)
    rel_chg_plot = plotRelChg(rel_chg[sname],sname)
    adf_dist_plot,adfma_dist_plot = plotDist(temp_t,sname)
    portValue.add_yaxis(
                    series_name="CTA_Value",
                    y_axis=round((temp_t['value']/1000),1).values.tolist(),
                    xaxis_index = 5,
                    yaxis_index = 7,
                    color = 'black',
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    )
    pnlScatter.add_yaxis( "CTA_Win", 0.001*temp_t.PnL_Win,
                    xaxis_index = 5,
                    yaxis_index = 8,
                    color = 'rgb(255, 0, 0)',
                    label_opts = opts.LabelOpts( is_show = False),
                    tooltip_opts=opts.TooltipOpts(is_show=False),
                )
    pnlScatter.add_yaxis( "CTA_Loss", 0.001*temp_t.PnL_Loss,
                    xaxis_index = 5,
                    yaxis_index = 8,
                    color = 'rgb(64, 191, 64)',
                    label_opts = opts.LabelOpts( is_show = False),
                    tooltip_opts=opts.TooltipOpts(is_show=False),
                )
    overlap  = portValue.overlap(pnlScatter)

    grid = Grid(
                init_opts=opts.InitOpts(
                width="1440px",
                height="600px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
            )
    ah_price_plot.set_colors( [  "rgb(172, 115, 58)",
                        "black",
                        "rgb(255, 102, 163)",
                        "rgb(121, 210, 121)",
                        "rgb(236, 198, 236)",
                        "grey",
                        'rgb(102, 102, 153)',
                        'black',
                        'brown',
                        'rgb(114, 140, 127)',
                        'rgb(152, 15, 15)',
                        'black',
                        'red',
                        'rgba(10, 162, 10, 0.6)',
                        'Salmon',
                        'green',
                        ]
                    )
    grid.add(ah_price_plot,grid_index = 0,grid_opts = opts.GridOpts(pos_right = '40%',pos_bottom = '50%'),is_control_axis_index = True)
    grid.add(ah_adf_plot,grid_index = 1,grid_opts = opts.GridOpts(pos_right = '40%',pos_top = '75%'),is_control_axis_index = True)
    grid.add(rel_chg_plot,grid_index = 2,grid_opts = opts.GridOpts(pos_left = '68%',pos_bottom = '35%'),is_control_axis_index = True)
    grid.add(adf_dist_plot,grid_index = 3,grid_opts = opts.GridOpts(pos_left = '65%',pos_right='23%',pos_top = '75%'),is_control_axis_index = True)
    grid.add(adfma_dist_plot,grid_index = 4,grid_opts = opts.GridOpts(pos_left = '80%',pos_right = '8%',pos_top = '75%'),is_control_axis_index = True)
    grid.add(overlap,grid_index = 5,grid_opts = opts.GridOpts(pos_right = '40%',pos_top = '70%',pos_bottom = '46%'),is_control_axis_index = True)
    # # grid.add(pnl_scatter,grid_index = 6,grid_opts = opts.GridOpts(pos_left = '65%',pos_top = '69%',pos_bottom = '46%'),is_control_axis_index = True)
    grid_tabs[sname] = grid
    #grid.render('report/'+sname+'_trend.html')

### Integrating into Grid

In [ ]:
banklist

In [ ]:
#stock_data['中信证券']
totalValue = None 
# finlist.remove('光大证券')
#banklist.remove('青岛银行')
#banklist.remove('郑州银行')

for nm in banklist:
    if totalValue is None:
        totalValue = stock_data[nm][['cash_OSCI','value_OSCI','cash_CTA','value_CTA']]
    else:
        totalValue = totalValue.join(stock_data[nm][['cash_OSCI','value_OSCI','cash_CTA','value_CTA']],how = 'outer',rsuffix='r')
        totalValue = totalValue.fillna(method = 'bfill')
        totalValue.loc[:,['cash_OSCI','value_OSCI','cash_CTA','value_CTA']] = totalValue.loc[:,['cash_OSCI','value_OSCI','cash_CTA','value_CTA']] +  totalValue.loc[:,['cash_OSCIr','value_OSCIr','cash_CTAr','value_CTAr']].values
        totalValue = totalValue[['cash_OSCI','value_OSCI','cash_CTA','value_CTA']]
totalValue.tail(10)
len(finlist+banklist)

In [ ]:
from pyecharts.charts import Bar, Grid, Line, Pie, Tab
tab = Tab()
for key in banklist:
    print(key)
    tab.add(grid_tabs[key], key)
tab.render('total.html')


In [ ]:
totalValue.tail(5)

In [ ]:
totalPortValue = (
        Line(
                init_opts=opts.InitOpts(
                width="1440px",
                height="500px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
        )
        .add_xaxis(totalValue.index.date.tolist())
        .add_yaxis(
                    series_name="CTA_Return",
                    y_axis=round(100*(totalValue['value_CTA']/totalValue['value_CTA'][0]),1).values.tolist(),
                    color = 'black',
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width = 2)
                    )
        .add_yaxis(
                    series_name="Osci_Return",
                    y_axis=round(100*(totalValue['value_OSCI']/totalValue['value_OSCI'][0]),1).values.tolist(),
                    color = 'rgb(174, 61, 23)',
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width = 2)
                    
                    )
        
        .extend_axis(
                    yaxis=opts.AxisOpts(
                                name = '资金利用率',
                                max_ = 200,
                                min_ = 'dataMin',
                                axislabel_opts=opts.LabelOpts(formatter="{value} %"),
                                splitline_opts=opts.SplitLineOpts(  is_show=False,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
                    )
        
        .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Return %',
                                max_ = 'dataMax',
                                min_ = 90,
                                axislabel_opts=opts.LabelOpts(formatter="{value} %"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
                            datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=False, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                            ],
                             tooltip_opts=opts.TooltipOpts(
                                trigger="axis",
                                axis_pointer_type="cross",
                                background_color="rgba(245, 245, 245, 0.8)",
                                border_width=1,
                                border_color="#ccc",
                                textstyle_opts=opts.TextStyleOpts(color="#000"),
                            ),
                            axispointer_opts=opts.AxisPointerOpts(
                                is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
                            ),
                        )
    )

totalCashUsage = (
    Bar()
    .add_xaxis(totalValue.index.date.tolist())
    .add_yaxis(
                series_name="Cash Usage CTA",
                y_axis=round(100*(totalValue['cash_CTA']/totalValue['value_CTA']),1).values.tolist(),
                color = 'black',
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                #itemstyle_opts = opts.ItemStyleOpts(width=1.5,type_ = 'solid',opacity=0.73, )
    )
    .add_yaxis(
                series_name="Cash Usage Osci",
                y_axis=round(100*(totalValue['cash_OSCI']/totalValue['value_OSCI']),1).values.tolist(),
                color = 'black',
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                #itemstyle_opts = opts.LineStyleOpts(width=1.5,type_ = 'solid',opacity=0.73,)
                )
    
    .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Usage%',
                                max_ = 200,
                                min_ = 0,
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
    )
)
totalPortValue.overlap(totalCashUsage)
totalPortValue.render_notebook()
tab.add(totalPortValue,'total')
tab.render('total.html')

In [ ]:
totalValue['cash_OSCI'][0]